In [13]:
import ipywidgets as widgets
from traitlets import Unicode, List, Int

In [14]:
widgets.__version__

'7.0.0'

# Example from http://bl.ocks.org/mbostock/4060366
# The above link has changed to v4 saved a slightly modified version in the same folder as index.html
http://bl.ocks.org/shawnbot/8059739

In [15]:
%%javascript

var s = document.createElement("style");
s.innerHTML = `
body {
  overflow: hidden;
}

path {
      fill-opacity: .8;
      stroke: #000;
}

path:first-child {
  fill: yellow !important;
}

circle {
      fill: #fff;
      stroke: #000;
      pointer-events: none;
}

    .PiYG .q0-9{fill:rgb(197,27,125)}
    .PiYG .q1-9{fill:rgb(222,119,174)}
    .PiYG .q2-9{fill:rgb(241,182,218)}
    .PiYG .q3-9{fill:rgb(253,224,239)}
    .PiYG .q4-9{fill:rgb(247,247,247)}
    .PiYG .q5-9{fill:rgb(230,245,208)}
    .PiYG .q6-9{fill:rgb(184,225,134)}
    .PiYG .q7-9{fill:rgb(127,188,65)}
    .PiYG .q8-9{fill:rgb(77,146,33)}
`;
document.getElementsByTagName("head")[0].appendChild(s);

<IPython.core.display.Javascript object>

In [16]:
class MyD3(widgets.DOMWidget):
    _view_name = Unicode('D3View').tag(sync=True)
    _view_module = Unicode('myd3').tag(sync=True)
    width = Int().tag(sync=True)
    height = Int().tag(sync=True)
    vertices = List().tag(sync=True)

In [75]:
%%javascript
require.undef('myd3');

define('myd3', ["@jupyter-widgets/base", //revised after upgrade to ipywidget 7.0s
                "http://d3js.org/d3.v3.min.js"], function(widgets, d3) {
    
    var D3View = widgets.DOMWidgetView.extend({
        
        render: function() {
                      
            var that = this;
            
            var width = this.model.get('width'),
                height = this.model.get('height'); //     var width = 960,height = 480;
            that.vertices = this.model.get('vertices'); //Math.random
            
            that.voronoi = d3.geom.voronoi()
                .clipExtent([[0, 0], [width, height]]);
            /*
            var voronoi = d3.geom.voronoi()
                .clipExtent([[0, 0], [width, height]]);
            */

            var svg = d3.select(this.el).append("svg") //var svg = d3.select("body").append("svg")
                .attr("width", width)                  //.attr("width", width)
                .attr("height", height)                //.attr("height", height)
                .attr("class", "PiYG")                 //.attr("class", "PiYG")
                .on("mousemove", function() {          //.on("mousemove", function() { 
                    that.vertices[0] = d3.mouse(this); //vertices[0] = d3.mouse(this);
                    that.redraw();                     //redraw();
                });                                    //});
            
            that.path = svg.append("g").selectAll("path"); //var path = svg.append("g").selectAll("path");

            that.line = d3.svg.line();                  //    var line = d3.svg.line();
            
            that.line.interpolate("linear");            //    line.interpolate(select.property("value"));
            
            that.circle = svg.append("g").selectAll("circle");  // None
            that.model.on('change:vertices', that.update_vertices, this);
            that.redraw();
        },
        
        update_vertices: function() {
            this.redraw();
        },
        
        redraw: function () {
          var that=this;
            
          this.vertices = this.model.get('vertices');
          //redraw the lines
          this.path = this.path
                          .data(this.voronoi(this.vertices), String);
          //path = path.data(voronoi(vertices), String);
          this.path.exit().remove(); //path.exit().remove();
            
          this.path.enter().append("path") //path.enter().append("path")
              .attr("class", function(d, i) { return "q" + (i % 9) + "-9"; }) //.attr("class", function(d, i) { return "q" + (i % 9) + "-9"; });
              .attr("d", function(d){  // path.attr("d", function(d) {
                return that.line(d)+'Z'; //return line(d) + "Z";
          });
          
          //this.path.order(); //      path.order();
            
          //redraw the circles  

          this.circle = this.circle.data([]);
          this.circle.exit().remove();         
          this.circle = this.circle
                            .data(this.vertices.slice(1));   

          this.circle.enter().append("circle")
                .attr("transform", function(d) { 
                        return "translate(" + d + ")"; 
                        })
                .attr("r", 2);
        },        
    });
    
    return {
        D3View : D3View
    };
});

<IPython.core.display.Javascript object>

In [76]:
import numpy as np
sample_size = 100
width = 750
height = 300

m = MyD3(vertices=(np.random.rand(sample_size, 2) * np.array([width, height])).tolist(),
     height=height, width=width)
m

A Jupyter Widget

In [79]:
m.vertices = (np.random.rand(sample_size, 2) * np.array([width, height])).tolist()